In [2]:
import pandas as pd
from sklearn.utils import shuffle

In [3]:
file = pd.read_csv("x_train_gr_smpl.csv", delimiter=',')
file.head()

,0,1,2,3,4,5,6,7,8,9,...,2294,2295,2296,2297,2298,2299,2300,2301,2302,2303
0,30.0,29.0,28.0,29.0,31.0,30.0,29.0,28.0,27.0,26.0,...,31.0,32.0,35.0,38.0,39.0,39.0,40.0,39.0,39.0,38.0
1,31.0,31.0,33.0,32.0,31.0,30.0,29.0,28.0,28.0,28.0,...,32.0,34.0,35.0,36.0,36.0,37.0,38.0,38.0,37.0,37.0
2,30.0,30.0,31.0,29.0,28.0,27.0,26.0,28.0,30.0,31.0,...,33.0,35.0,37.0,37.0,38.0,39.0,38.0,38.0,39.0,40.0
3,26.0,25.0,24.0,24.0,24.0,27.0,28.0,29.0,29.0,30.0,...,32.0,34.0,36.0,37.0,38.0,42.0,40.0,37.0,36.0,36.0
4,25.0,26.0,28.0,28.0,28.0,28.0,28.0,27.0,26.0,25.0,...,30.0,31.0,33.0,37.0,38.0,37.0,36.0,36.0,35.0,35.0


In [4]:
Y = pd.read_csv("y_train_smpl.csv", delimiter=',')
Y.columns = ['target']
Y.tail()

,target
12655,9
12656,9
12657,9
12658,9
12659,9


In [5]:
data = pd.concat([file, Y], axis=1)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,2295,2296,2297,2298,2299,2300,2301,2302,2303,target
0,30.0,29.0,28.0,29.0,31.0,30.0,29.0,28.0,27.0,26.0,...,32.0,35.0,38.0,39.0,39.0,40.0,39.0,39.0,38.0,0
1,31.0,31.0,33.0,32.0,31.0,30.0,29.0,28.0,28.0,28.0,...,34.0,35.0,36.0,36.0,37.0,38.0,38.0,37.0,37.0,0
2,30.0,30.0,31.0,29.0,28.0,27.0,26.0,28.0,30.0,31.0,...,35.0,37.0,37.0,38.0,39.0,38.0,38.0,39.0,40.0,0
3,26.0,25.0,24.0,24.0,24.0,27.0,28.0,29.0,29.0,30.0,...,34.0,36.0,37.0,38.0,42.0,40.0,37.0,36.0,36.0,0
4,25.0,26.0,28.0,28.0,28.0,28.0,28.0,27.0,26.0,25.0,...,31.0,33.0,37.0,38.0,37.0,36.0,36.0,35.0,35.0,0


In [6]:
data = shuffle(data)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,2295,2296,2297,2298,2299,2300,2301,2302,2303,target
4254,37.0,31.0,31.0,33.0,34.0,31.0,23.0,19.0,20.0,21.0,...,60.0,60.0,56.0,57.0,56.0,61.0,68.0,69.0,92.0,3
10541,84.0,86.0,87.0,75.0,66.0,62.0,83.0,104.0,126.0,122.0,...,51.0,63.0,71.0,74.0,76.0,80.0,86.0,96.0,105.0,8
1648,11.0,11.0,13.0,14.0,13.0,14.0,15.0,15.0,15.0,15.0,...,10.0,11.0,11.0,11.0,10.0,10.0,10.0,8.0,9.0,1
10925,254.0,254.0,255.0,254.0,254.0,252.0,248.0,242.0,239.0,242.0,...,72.0,75.0,77.0,78.0,77.0,78.0,82.0,86.0,90.0,8
590,17.0,14.0,14.0,13.0,13.0,15.0,16.0,14.0,13.0,12.0,...,22.0,20.0,21.0,19.0,17.0,17.0,17.0,18.0,17.0,0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12660 entries, 4254 to 9957
Columns: 2305 entries, 0 to target
dtypes: float64(2304), int64(1)
memory usage: 222.7 MB


In [8]:
data.describe()

,0,1,2,3,4,5,6,7,8,9,...,2295,2296,2297,2298,2299,2300,2301,2302,2303,target
count,12660.000000,12660.000000,12660.000000,12660.000000,12660.000000,12660.000000,12660.000000,12660.000000,12660.000000,12660.000000,...,12660.000000,12660.000000,12660.000000,12660.000000,12660.000000,12660.000000,12660.000000,12660.000000,12660.000000,12660.000000
mean,90.275908,90.296840,90.331043,90.369510,90.608847,90.852370,91.113033,91.427488,91.566588,91.775355,...,71.547393,71.399368,70.980253,70.064455,69.243681,68.339336,67.716983,67.734992,67.784123,4.066351
std,79.531811,79.674242,79.816240,79.896452,79.958141,80.082173,80.266651,80.406439,80.439131,80.457116,...,65.317373,65.485237,65.649521,65.109495,64.328815,63.312147,62.688005,62.951496,63.289188,2.670880
min,5.000000,6.000000,5.000000,6.000000,6.000000,6.000000,6.000000,6.000000,5.000000,6.000000,...,6.000000,6.000000,6.000000,5.000000,6.000000,6.000000,6.000000,6.000000,5.000000,0.000000
25%,29.000000,29.000000,29.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,29.000000,...,25.000000,25.000000,25.000000,25.000000,24.000000,24.000000,24.000000,24.000000,24.000000,1.000000
50%,55.000000,56.000000,55.000000,55.000000,56.000000,56.000000,56.000000,56.000000,57.000000,57.000000,...,47.000000,47.000000,46.000000,45.000000,44.000000,44.000000,44.000000,43.000000,43.000000,4.000000
75%,136.000000,136.000000,135.000000,136.000000,137.000000,137.000000,140.000000,140.000000,140.000000,140.000000,...,93.000000,93.000000,92.000000,91.000000,90.000000,88.000000,87.000000,86.000000,87.000000,6.000000
max,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,...,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,9.000000


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data[data.columns[:2303]], data['target'], test_size=0.33, random_state=42)

In [12]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.47103877453326953